# Yelp Business dataset


## Columns
- 'address'
-  'attributes'
-  'business_id'
-  'categories'
-  'city'
-  'hours'
-  'is_open'
-  'latitude'
-  'longitude'
-  'name
- 'neighborhood'
-  'postal_code'
-  'review_count'
-  'stars'
- 'state'

## Total Business
- 188593
### States
- 69

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from collections import OrderedDict
from operator import itemgetter
import re

In [2]:
ls yelp_data/

yelp_academic_dataset_business.json*  yelp_academic_dataset_review.json*


In [3]:
path = 'yelp_data/yelp_academic_dataset_business.json'

In [4]:
%%time
df = pd.read_json(path, lines=True)

CPU times: user 6 s, sys: 3.75 s, total: 9.75 s
Wall time: 9.74 s


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188593 entries, 0 to 188592
Data columns (total 15 columns):
address         188593 non-null object
attributes      162807 non-null object
business_id     188593 non-null object
categories      188052 non-null object
city            188593 non-null object
hours           143791 non-null object
is_open         188593 non-null int64
latitude        188587 non-null float64
longitude       188587 non-null float64
name            188593 non-null object
neighborhood    188593 non-null object
postal_code     188593 non-null object
review_count    188593 non-null int64
stars           188593 non-null float64
state           188593 non-null object
dtypes: float64(3), int64(2), object(10)
memory usage: 21.6+ MB


In [6]:
len(df.business_id.unique())

188593

In [7]:
len(df.state.unique())

69

In [8]:
df.state.value_counts()

AZ     56495
NV     35688
ON     32393
NC     14359
OH     13664
PA     10966
QC      8756
AB      7670
WI      5042
IL      1937
SC       770
NYK      163
NI       134
IN       101
OR        72
BY        60
ST        45
CO        43
C         34
HE        32
XGM       23
NLK       23
NY        19
RP        19
01        11
CA         8
VS         7
NW         4
O          3
6          3
       ...  
G          2
KY         1
HH         1
GA         1
4          1
DE         1
CS         1
PO         1
V          1
10         1
HU         1
CC         1
LU         1
MT         1
45         1
TAM        1
CRF        1
11         1
RCC        1
WHT        1
MN         1
AG         1
BC         1
MA         1
VA         1
NE         1
WAR        1
SP         1
XMS        1
FL         1
Name: state, Length: 69, dtype: int64

# Select columns

Select 
- business_id
- name
- categories
- state

In [9]:
#list(df.columns)
df = df[[
 'business_id',
 'categories',
 'name','state']]

In [10]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Action Engine,AB


## Drop null categories

- we need to drop any rows with no categories

In [11]:
len(df[df.categories.isnull()])

541

In [12]:
#df[df.categories.isnull()]

### Create null categorie filter

In [13]:
null_cat = df.categories.isnull()

In [14]:
len(df[~null_cat])

188052

In [15]:
df = df[~null_cat] # display rows NOT in null_cat filter

### Reset index after filter

In [16]:
df = df.reset_index(drop=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188052 entries, 0 to 188051
Data columns (total 4 columns):
business_id    188052 non-null object
categories     188052 non-null object
name           188052 non-null object
state          188052 non-null object
dtypes: object(4)
memory usage: 5.7+ MB


In [18]:
df.head()

,business_id,categories,name,state
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Minhas Micro Brewery,AB
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",CK'S BBQ & Catering,NV
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",La Bastringue,QC
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Geico Insurance,AZ
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Action Engine,AB


## Clean up Categories column
- categories is a list of ... duh categories
-  we will lowercase categories
- convert each categories ithem into a list by splitting on a comma

In [19]:
df.categories = df.categories.str.lower()

## duplicate categories
- duplicate categories as a new col called 'categories_list'
- split on comma

In [20]:
df['categories_list'] = df.categories
df['categories_list']  = df['categories_list'].str.split(',')

In [21]:
df.head()

,business_id,categories,name,state,categories_list
0,Apn5Q_b6Nz61Tq4XzPdf9A,"tours, breweries, pizza, restaurants, food, ho...",Minhas Micro Brewery,AB,"[tours, breweries, pizza, restaurants, foo..."
1,AjEbIBw6ZFfln7ePHha9PA,"chicken wings, burgers, caterers, street vendo...",CK'S BBQ & Catering,NV,"[chicken wings, burgers, caterers, street v..."
2,O8S5hYJ1SMc8fA4QBtVujA,"breakfast & brunch, restaurants, french, sandw...",La Bastringue,QC,"[breakfast & brunch, restaurants, french, s..."
3,bFzdJJ3wp3PZssNEsyU23g,"insurance, financial services",Geico Insurance,AZ,"[insurance, financial services]"
4,8USyCYqpScwiNEb58Bt6CA,"home & garden, nurseries & gardening, shopping...",Action Engine,AB,"[home & garden, nurseries & gardening, shopp..."


# Count unique categories
- count how many times each categorie shows up
- looping over  df.categories_list
- create a dict with name of the categories as key, and total count as values

In [22]:
%%time
counts = dict()
for i in range(len(df.categories_list)):
    for j in df.categories_list[i]:
        j= j.lstrip()
        counts[j] = counts.get(j, 0) + 1

CPU times: user 8.16 s, sys: 0 ns, total: 8.16 s
Wall time: 8.16 s


## total categories
- 1305

In [23]:
len(counts)

1305

In [24]:
type(counts)

dict

## Create a dataframe from our new categories dictionary for better understanding

In [25]:
cat_reviews = pd.DataFrame.from_dict(counts,columns=['total_reviews'],orient='index')

In [27]:
cat_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1305 entries, tours to japanese sweets
Data columns (total 1 columns):
total_reviews    1305 non-null int64
dtypes: int64(1)
memory usage: 20.4+ KB


In [28]:
cat_reviews.head()

,total_reviews
tours,639
breweries,508
pizza,6603
restaurants,57173
food,27118


## Sort dataframe by total_reviews

In [29]:
cat_reviews.sort_values(by=['total_reviews'], ascending=False, inplace=True)

In [31]:
cat_reviews.head()

,total_reviews
restaurants,57173
shopping,30231
food,27118
beauty & spas,18967
home services,18634


## Create a list with Healthcare related categories

In [34]:
healthcare = ['doctors',
           'family practice',
           'urgent care',
           'obstetricians & gynecologists',
          'cosmetic surgeons',
           'internal medicine',
           'dermatologists',
           'surgeons',
           'ear nose & throat',
           'psychiatrists',
           'oncologist',
           'plastic surgeons',
           'gastroenterologist',
           'urologists',
           'emergency rooms',
           'pulmonologist',
           'spine surgeons',
           'pathologists',
           'neurotologists',
           'emergency medicine',
           'ultrasound imaging centers'
          ]

## Create new dataframe
- use our new healthcare categories list
- applying a lambda function on df.categories
    - spliting on comma
    - and searching if ithem is in our list

In [35]:
%%time
health = df[df['categories'].apply(lambda x: pd.Series(x.split(', ')).isin(healthcare).any())]

CPU times: user 1min 14s, sys: 46.9 ms, total: 1min 14s
Wall time: 1min 14s


In [37]:
health.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5656 entries, 78 to 187983
Data columns (total 5 columns):
business_id        5656 non-null object
categories         5656 non-null object
name               5656 non-null object
state              5656 non-null object
categories_list    5656 non-null object
dtypes: object(5)
memory usage: 265.1+ KB


In [42]:
health = health.reset_index(drop=True)  # reset index

In [43]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5656 entries, 0 to 5655
Data columns (total 5 columns):
business_id        5656 non-null object
categories         5656 non-null object
name               5656 non-null object
state              5656 non-null object
categories_list    5656 non-null object
dtypes: object(5)
memory usage: 221.0+ KB


## Review

### Total Healthcare  business
- 5656

- save our new healthcare business dataframe

In [46]:
health.to_csv('yelp_data/health_business00.csv')

In [49]:
ls yelp_data/

health_business00.csv*                yelp_academic_dataset_review.json*
yelp_academic_dataset_business.json*
